#### Build a Simple LLM Application with LECL
In this quickstrat we'll show you how to build a simple LLM application with LangChain.This application will
translate text from English into another language.This is a relatively simple LLM application - it's just a single
LLM plus some prompting. Still, this is a great way to get strated with Langchain- a lot of features can be built
with just some prompting and an LLM call!

You will learn from this video:
*  Using language models
*  Using PromptTemplates and OutputParsers
* Using LangChain Expression Language(LCEL) to chain components together
* Debugging and tracing your application using LangSmith
* Deploy your application with LangServe

In [1]:
### Open API and Open Source models -- llama3, Gemma2, mistral, etc.. --groq

import os
from dotenv import load_dotenv

load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")


In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It", api_key=groq_api_key)
model

v:\Genai_course\Langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002BC547693A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002BC543900B0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following English text to French."),
    HumanMessage(content="Hello, how are you?"),
]

result = model.invoke(messages)

In [8]:
result

AIMessage(content='Bonjour, comment allez-vous ? \n\n\nYou can also use the more informal:\n\n* Salut, comment vas-tu ? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 23, 'total_tokens': 53, 'completion_time': 0.054545455, 'prompt_time': 0.001351258, 'queue_time': 0.253509192, 'total_time': 0.055896713}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--a7736d84-aa86-49dc-88b4-9bbf616775fb-0', usage_metadata={'input_tokens': 23, 'output_tokens': 30, 'total_tokens': 53})

In [9]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(result)

'Bonjour, comment allez-vous ? \n\n\nYou can also use the more informal:\n\n* Salut, comment vas-tu ? \n'

In [10]:
### Using LCEL - we can chain the components together
chain = model | parser
chain.invoke(messages)

'Bonjour, comment allez-vous ? \n\n\nYou can also use a more informal version:\n\n**Salut, ça va ?** \n'

In [12]:
### Prompt templates
from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate the following into {language}: "

prompt = ChatPromptTemplate.from_messages(
    [("system",generic_template),("human", "{text}")]
)


In [14]:
result = prompt.invoke({"language":"French","text":"Hello"})

In [15]:
result.to_messages()

[SystemMessage(content='Translate the following into French: ', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [16]:
chain = prompt | model | parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'